<a href="https://colab.research.google.com/github/countifs/kaggle/blob/main/L%26T_Vehicle_Loan_Default_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 작업환경 설정**

> ## **1.1 런타임 유형 변경 : None → GPU**

In [ ]:
! cat /proc/cpuinfo

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



> ## **1.2 구글 드라이브 마운트**

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


> ## **1.3 작업경로 변경**

In [ ]:
print('> 변경 전 경로 : ' , os.getcwd())
os.chdir('/content/drive/MyDrive/kaggle/2. L&T Vehicle Loan Default Prediction')
print('> 변경 후 경로 : ' , os.getcwd())

> 변경 전 경로 :  /content
> 변경 후 경로 :  /content/drive/MyDrive/kaggle/2. L&T Vehicle Loan Default Prediction


> ## **1.4 라이브러리 불러오기**

In [ ]:
#load packages

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import random as rn

# loading Common Model Algorithms

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_digits
from sklearn import tree

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# **2. 데이터 준비**

> ## **2.1 데이터 불러오기**

In [113]:
train_df= pd.read_csv('train.csv')
test_df= pd.read_csv('test.csv')
variable_df = pd.read_csv('data_dictionary.csv')

 > ## **2.2 변수 설명**

In [114]:
#컬럼명 조회
print(type(variable_df.columns))
print(variable_df.columns)

<class 'pandas.core.indexes.base.Index'>
Index(['Unnamed: 0', 'Variable Name', 'Description', 'Unnamed: 2'], dtype='object')


In [115]:
#컬럼명 조회 (리스트를 시리즈로 변환 list to Series)
variable_df_columns = pd.Series(variable_df.columns)
variable_df_columns

0       Unnamed: 0
1    Variable Name
2      Description
3       Unnamed: 2
dtype: object

In [116]:
#컬럼명 조회 (리스트를 데이터프레임으로 변환 list to DataFrame)
variable_df_columns2 = pd.DataFrame([variable_df.columns])
variable_df_columns2.T

,0
0,Unnamed: 0
1,Variable Name
2,Description
3,Unnamed: 2


In [117]:
#변수정보 출력
print(type(variable_df))
variable_df.T 

<class 'pandas.core.frame.DataFrame'>


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
Variable Name,UniqueID,loan_default,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
Description,Identifier for customers,Payment default in the first EMI on due date,Amount of Loan disbursed,Cost of the Asset,Loan to Value of the asset,Branch where the loan was disbursed,Vehicle Dealer where the loan was disbursed,"Vehicle manufacturer(Hero, Honda, TVS etc.)",Current pincode of the customer,Date of birth of the customer,Employment Type of the customer (Salaried/Self...,Date of disbursement,State of disbursement,Employee of the organization who logged the di...,if Mobile no. was shared by the customer then ...,if aadhar was shared by the customer then flag...,if pan was shared by the customer then flagged...,if voter was shared by the customer then flag...,if DL was shared by the customer then flagged ...,if passport was shared by the customer then fl...,Bureau Score,Bureau score description,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,EMI Amount of the primary loan,EMI Amount of the secondary loan,New loans taken by the customer in last 6 mont...,Loans defaulted in the last 6 months,Average loan tenure,Time since first loan,Enquries done by the customer for loans
Unnamed: 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Primary accounts are those which the customer ...,NaN,NaN,NaN,NaN,NaN,Secondary accounts are those which the custome...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 >> ### **2.2.1 변수설명 테이블 정리**

In [118]:
#불필요 컬럼 제거("drop" + "컬럼명" + "axis=1" 을 통한 불필요 컬럼 제거)
variable_df1 = variable_df.drop(['Unnamed: 0','Unnamed: 2'], axis = 1) 
variable_df1.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
Variable Name,UniqueID,loan_default,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
Description,Identifier for customers,Payment default in the first EMI on due date,Amount of Loan disbursed,Cost of the Asset,Loan to Value of the asset,Branch where the loan was disbursed,Vehicle Dealer where the loan was disbursed,"Vehicle manufacturer(Hero, Honda, TVS etc.)",Current pincode of the customer,Date of birth of the customer,Employment Type of the customer (Salaried/Self...,Date of disbursement,State of disbursement,Employee of the organization who logged the di...,if Mobile no. was shared by the customer then ...,if aadhar was shared by the customer then flag...,if pan was shared by the customer then flagged...,if voter was shared by the customer then flag...,if DL was shared by the customer then flagged ...,if passport was shared by the customer then fl...,Bureau Score,Bureau score description,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,EMI Amount of the primary loan,EMI Amount of the secondary loan,New loans taken by the customer in last 6 mont...,Loans defaulted in the last 6 months,Average loan tenure,Time since first loan,Enquries done by the customer for loans


In [119]:
#불필요 컬럼 제거("drop" + "columns" 통한 불필요 컬럼 제거)
variable_df5 = variable_df.drop(columns = ['Unnamed: 0','Unnamed: 2'])
variable_df5.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
Variable Name,UniqueID,loan_default,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
Description,Identifier for customers,Payment default in the first EMI on due date,Amount of Loan disbursed,Cost of the Asset,Loan to Value of the asset,Branch where the loan was disbursed,Vehicle Dealer where the loan was disbursed,"Vehicle manufacturer(Hero, Honda, TVS etc.)",Current pincode of the customer,Date of birth of the customer,Employment Type of the customer (Salaried/Self...,Date of disbursement,State of disbursement,Employee of the organization who logged the di...,if Mobile no. was shared by the customer then ...,if aadhar was shared by the customer then flag...,if pan was shared by the customer then flagged...,if voter was shared by the customer then flag...,if DL was shared by the customer then flagged ...,if passport was shared by the customer then fl...,Bureau Score,Bureau score description,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,EMI Amount of the primary loan,EMI Amount of the secondary loan,New loans taken by the customer in last 6 mont...,Loans defaulted in the last 6 months,Average loan tenure,Time since first loan,Enquries done by the customer for loans


In [120]:
#불필요 컬럼 제거("drop" + "columns" 통한 불필요 컬럼 제거)
variable_df6 = variable_df.drop(columns = [variable_df.columns[0],variable_df.columns[3]])
variable_df6.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
Variable Name,UniqueID,loan_default,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
Description,Identifier for customers,Payment default in the first EMI on due date,Amount of Loan disbursed,Cost of the Asset,Loan to Value of the asset,Branch where the loan was disbursed,Vehicle Dealer where the loan was disbursed,"Vehicle manufacturer(Hero, Honda, TVS etc.)",Current pincode of the customer,Date of birth of the customer,Employment Type of the customer (Salaried/Self...,Date of disbursement,State of disbursement,Employee of the organization who logged the di...,if Mobile no. was shared by the customer then ...,if aadhar was shared by the customer then flag...,if pan was shared by the customer then flagged...,if voter was shared by the customer then flag...,if DL was shared by the customer then flagged ...,if passport was shared by the customer then fl...,Bureau Score,Bureau score description,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,EMI Amount of the primary loan,EMI Amount of the secondary loan,New loans taken by the customer in last 6 mont...,Loans defaulted in the last 6 months,Average loan tenure,Time since first loan,Enquries done by the customer for loans


In [121]:
# 출력할 컬럼만 입력해서 조회 ("컬럼명 입력")
variable_df2 = variable_df[['Variable Name', 'Description']]
variable_df2.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
Variable Name,UniqueID,loan_default,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
Description,Identifier for customers,Payment default in the first EMI on due date,Amount of Loan disbursed,Cost of the Asset,Loan to Value of the asset,Branch where the loan was disbursed,Vehicle Dealer where the loan was disbursed,"Vehicle manufacturer(Hero, Honda, TVS etc.)",Current pincode of the customer,Date of birth of the customer,Employment Type of the customer (Salaried/Self...,Date of disbursement,State of disbursement,Employee of the organization who logged the di...,if Mobile no. was shared by the customer then ...,if aadhar was shared by the customer then flag...,if pan was shared by the customer then flagged...,if voter was shared by the customer then flag...,if DL was shared by the customer then flagged ...,if passport was shared by the customer then fl...,Bureau Score,Bureau score description,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,EMI Amount of the primary loan,EMI Amount of the secondary loan,New loans taken by the customer in last 6 mont...,Loans defaulted in the last 6 months,Average loan tenure,Time since first loan,Enquries done by the customer for loans


In [122]:
#iloc는 numeric 컬럼 인덱스 요구
variable_df3 = variable_df.iloc[:, 1:3] 
variable_df3.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
Variable Name,UniqueID,loan_default,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
Description,Identifier for customers,Payment default in the first EMI on due date,Amount of Loan disbursed,Cost of the Asset,Loan to Value of the asset,Branch where the loan was disbursed,Vehicle Dealer where the loan was disbursed,"Vehicle manufacturer(Hero, Honda, TVS etc.)",Current pincode of the customer,Date of birth of the customer,Employment Type of the customer (Salaried/Self...,Date of disbursement,State of disbursement,Employee of the organization who logged the di...,if Mobile no. was shared by the customer then ...,if aadhar was shared by the customer then flag...,if pan was shared by the customer then flagged...,if voter was shared by the customer then flag...,if DL was shared by the customer then flagged ...,if passport was shared by the customer then fl...,Bureau Score,Bureau score description,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,EMI Amount of the primary loan,EMI Amount of the secondary loan,New loans taken by the customer in last 6 mont...,Loans defaulted in the last 6 months,Average loan tenure,Time since first loan,Enquries done by the customer for loans


In [123]:
#loc는 string 컬럼명 값 요구
variable_df4 = variable_df.loc[:,['Variable Name','Description']] 
variable_df4.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
Variable Name,UniqueID,loan_default,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
Description,Identifier for customers,Payment default in the first EMI on due date,Amount of Loan disbursed,Cost of the Asset,Loan to Value of the asset,Branch where the loan was disbursed,Vehicle Dealer where the loan was disbursed,"Vehicle manufacturer(Hero, Honda, TVS etc.)",Current pincode of the customer,Date of birth of the customer,Employment Type of the customer (Salaried/Self...,Date of disbursement,State of disbursement,Employee of the organization who logged the di...,if Mobile no. was shared by the customer then ...,if aadhar was shared by the customer then flag...,if pan was shared by the customer then flagged...,if voter was shared by the customer then flag...,if DL was shared by the customer then flagged ...,if passport was shared by the customer then fl...,Bureau Score,Bureau score description,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,count of total loans taken by the customer at ...,count of active loans taken by the customer at...,count of default accounts at the time of disbu...,total Principal outstanding amount of the acti...,total amount that was sanctioned for all the l...,total amount that was disbursed for all the lo...,EMI Amount of the primary loan,EMI Amount of the secondary loan,New loans taken by the customer in last 6 mont...,Loans defaulted in the last 6 months,Average loan tenure,Time since first loan,Enquries done by the customer for loans


In [124]:
# 첫번째 열을 제외하고 변수 설명 데이터를 추출
variable_df7 = variable_df.iloc[:, 1:4]

In [125]:
# 엑셀로 저장 (엑셀 저장관련 공부 필요  ex)중복파일 저장 처리, 엑셀 서식 관련 처리 방법 등)
variable_df7.to_excel("variable_df7.xlsx")

 > ### **2.2.2 데이터셋 기본 정보 확인하기**

In [126]:
train_df.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,03-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,26-09-18,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,01-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,26-10-18,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,26-09-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


In [127]:
train_df.columns

Index(['UniqueID', 'disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID', 'Date.of.Birth',
       'Employment.Type', 'DisbursalDate', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS',
       'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES',
       'loan_default'],
      dtype='object')

 > #### **2.2.2.1 컬럼명 인덱스 번호 삽입**

In [128]:
from pprint import pprint

sub_columns = []

for i in range(len(train_df.columns)):
    columns_index = str(i+1)+"_"
    columns_names = train_df.columns[i]
    index_names = columns_index + columns_names
    sub_columns.append(index_names)

pprint(sub_columns)


['1_UniqueID',
 '2_disbursed_amount',
 '3_asset_cost',
 '4_ltv',
 '5_branch_id',
 '6_supplier_id',
 '7_manufacturer_id',
 '8_Current_pincode_ID',
 '9_Date.of.Birth',
 '10_Employment.Type',
 '11_DisbursalDate',
 '12_State_ID',
 '13_Employee_code_ID',
 '14_MobileNo_Avl_Flag',
 '15_Aadhar_flag',
 '16_PAN_flag',
 '17_VoterID_flag',
 '18_Driving_flag',
 '19_Passport_flag',
 '20_PERFORM_CNS.SCORE',
 '21_PERFORM_CNS.SCORE.DESCRIPTION',
 '22_PRI.NO.OF.ACCTS',
 '23_PRI.ACTIVE.ACCTS',
 '24_PRI.OVERDUE.ACCTS',
 '25_PRI.CURRENT.BALANCE',
 '26_PRI.SANCTIONED.AMOUNT',
 '27_PRI.DISBURSED.AMOUNT',
 '28_SEC.NO.OF.ACCTS',
 '29_SEC.ACTIVE.ACCTS',
 '30_SEC.OVERDUE.ACCTS',
 '31_SEC.CURRENT.BALANCE',
 '32_SEC.SANCTIONED.AMOUNT',
 '33_SEC.DISBURSED.AMOUNT',
 '34_PRIMARY.INSTAL.AMT',
 '35_SEC.INSTAL.AMT',
 '36_NEW.ACCTS.IN.LAST.SIX.MONTHS',
 '37_DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
 '38_AVERAGE.ACCT.AGE',
 '39_CREDIT.HISTORY.LENGTH',
 '40_NO.OF_INQUIRIES',
 '41_loan_default']


In [131]:
#컬럼명 변경
train_df.columns = sub_columns

In [132]:
train_df.head()

,1_UniqueID,2_disbursed_amount,3_asset_cost,4_ltv,5_branch_id,6_supplier_id,7_manufacturer_id,8_Current_pincode_ID,9_Date.of.Birth,10_Employment.Type,11_DisbursalDate,12_State_ID,13_Employee_code_ID,14_MobileNo_Avl_Flag,15_Aadhar_flag,16_PAN_flag,17_VoterID_flag,18_Driving_flag,19_Passport_flag,20_PERFORM_CNS.SCORE,21_PERFORM_CNS.SCORE.DESCRIPTION,22_PRI.NO.OF.ACCTS,23_PRI.ACTIVE.ACCTS,24_PRI.OVERDUE.ACCTS,25_PRI.CURRENT.BALANCE,26_PRI.SANCTIONED.AMOUNT,27_PRI.DISBURSED.AMOUNT,28_SEC.NO.OF.ACCTS,29_SEC.ACTIVE.ACCTS,30_SEC.OVERDUE.ACCTS,31_SEC.CURRENT.BALANCE,32_SEC.SANCTIONED.AMOUNT,33_SEC.DISBURSED.AMOUNT,34_PRIMARY.INSTAL.AMT,35_SEC.INSTAL.AMT,36_NEW.ACCTS.IN.LAST.SIX.MONTHS,37_DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,38_AVERAGE.ACCT.AGE,39_CREDIT.HISTORY.LENGTH,40_NO.OF_INQUIRIES,41_loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,03-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,26-09-18,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,01-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,26-10-18,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,26-09-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


 > #### **2.2.2.2 데이터셋 information 확인**

In [133]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233154 entries, 0 to 233153
Data columns (total 41 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   1_UniqueID                              233154 non-null  int64  
 1   2_disbursed_amount                      233154 non-null  int64  
 2   3_asset_cost                            233154 non-null  int64  
 3   4_ltv                                   233154 non-null  float64
 4   5_branch_id                             233154 non-null  int64  
 5   6_supplier_id                           233154 non-null  int64  
 6   7_manufacturer_id                       233154 non-null  int64  
 7   8_Current_pincode_ID                    233154 non-null  int64  
 8   9_Date.of.Birth                         233154 non-null  object 
 9   10_Employment.Type                      225493 non-null  object 
 10  11_DisbursalDate                        2331

In [134]:
train_df.describe()

,1_UniqueID,2_disbursed_amount,3_asset_cost,4_ltv,5_branch_id,6_supplier_id,7_manufacturer_id,8_Current_pincode_ID,12_State_ID,13_Employee_code_ID,14_MobileNo_Avl_Flag,15_Aadhar_flag,16_PAN_flag,17_VoterID_flag,18_Driving_flag,19_Passport_flag,20_PERFORM_CNS.SCORE,22_PRI.NO.OF.ACCTS,23_PRI.ACTIVE.ACCTS,24_PRI.OVERDUE.ACCTS,25_PRI.CURRENT.BALANCE,26_PRI.SANCTIONED.AMOUNT,27_PRI.DISBURSED.AMOUNT,28_SEC.NO.OF.ACCTS,29_SEC.ACTIVE.ACCTS,30_SEC.OVERDUE.ACCTS,31_SEC.CURRENT.BALANCE,32_SEC.SANCTIONED.AMOUNT,33_SEC.DISBURSED.AMOUNT,34_PRIMARY.INSTAL.AMT,35_SEC.INSTAL.AMT,36_NEW.ACCTS.IN.LAST.SIX.MONTHS,37_DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,40_NO.OF_INQUIRIES,41_loan_default
count,233154.000000,233154.000000,2.331540e+05,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.0,233154.00000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,233154.000000,2.331540e+05,2.331540e+05,2.331540e+05,233154.000000,233154.000000,233154.000000,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,2.331540e+05,233154.000000,233154.000000,233154.000000,233154.000000
mean,535917.573376,54356.993528,7.586507e+04,74.746530,72.936094,19638.635035,69.028054,3396.880247,7.262243,1549.477148,1.0,0.84032,0.075577,0.144943,0.023242,0.002127,289.462994,2.440636,1.039896,0.156549,1.659001e+05,2.185039e+05,2.180659e+05,0.059081,0.027703,0.007244,5.427793e+03,7.295923e+03,7.179998e+03,1.310548e+04,3.232684e+02,0.381833,0.097481,0.206615,0.217071
std,68315.693711,12971.314171,1.894478e+04,11.456636,69.834995,3491.949566,22.141304,2238.147502,4.482230,975.261278,0.0,0.36631,0.264320,0.352044,0.150672,0.046074,338.374779,5.217233,1.941496,0.548787,9.422736e+05,2.374794e+06,2.377744e+06,0.626795,0.316057,0.111079,1.702370e+05,1.831560e+05,1.825925e+05,1.513679e+05,1.555369e+04,0.955107,0.384439,0.706498,0.412252
min,417428.000000,13320.000000,3.700000e+04,10.030000,1.000000,10524.000000,45.000000,1.000000,1.000000,1.000000,1.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.678296e+06,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,-5.746470e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,476786.250000,47145.000000,6.571700e+04,68.880000,14.000000,16535.000000,48.000000,1511.000000,4.000000,713.000000,1.0,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,535978.500000,53803.000000,7.094600e+04,76.800000,61.000000,20333.000000,86.000000,2970.000000,6.000000,1451.000000,1.0,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,595039.750000,60413.000000,7.920175e+04,83.670000,130.000000,23000.000000,86.000000,5677.000000,10.000000,2362.000000,1.0,1.00000,0.000000,0.000000,0.000000,0.000000,678.000000,3.000000,1.000000,0.000000,3.500650e+04,6.250000e+04,6.080000e+04,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.999000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000
max,671084.000000,990572.000000,1.628992e+06,95.000000,261.000000,24803.000000,156.000000,7345.000000,22.000000,3795.000000,1.0,1.00000,1.000000,1.000000,1.000000,1.000000,890.000000,453.000000,144.000000,25.000000,9.652492e+07,1.000000e+09,1.000000e+09,52.000000,36.000000,8.000000,3.603285e+07,3.000000e+07,3.000000e+07,2.564281e+07,4.170901e+06,35.000000,20.000000,36.000000,1.000000
